In [14]:
import numpy as np
import pickle
import json
import os
import sys
import copy
import sklearn.preprocessing
import models
from sklearn.neighbors import NearestNeighbors
import utils

In [2]:
DATA_DIR = "../../data/fb15k/"
DUMP_FILE = "../dumps/fb15k_distmult_dump_norm.pkl"
MODEL_TYPE = "distmult"

In [3]:
train_data = utils.read_data(os.path.join(DATA_DIR,"train.txt"))
# dev_data = read_data(os.path.join(DATA_DIR,"valid.txt"))
# test_data = read_data(os.path.join(DATA_DIR,"test.txt"))

In [4]:
dump=utils.load_pickle(DUMP_FILE)
dump.keys()

dict_keys(['tail_rel_type', 'entity_type', 'entity_to_id', 'head_rel_type', 'entity_real', 'rel_real', 'relation_to_id'])

In [5]:
model=models.TypedDM(DUMP_FILE)

In [6]:
mapped_train_data = utils.map_data(train_data,dump)
# mapped_dev_data = map_data(dev_data)
# mapped_test_data = map_data(test_data)

In [7]:
entity_to_rel=utils.get_ent_to_rel(mapped_train_data)

In [30]:
rules_1_path="./pruned_rules_1.pkl"
rules_2_path="./pruned_rules_2.pkl"
rules_1=utils.load_pickle(rules_1_path)
rules_2=utils.load_pickle(rules_2_path)

In [9]:
def add_relation_body(rules,relation_to_body):
    for rule in rules:
        if rule[1] not in relation_to_body:
            relation_to_body[rule[1]]=[]
        relation_to_body[rule[1]].append(rule[0])
    return relation_to_body

In [31]:
relation_to_body={}
relation_to_body=add_relation_body(rules_1,relation_to_body)
relation_to_body=add_relation_body(rules_2,relation_to_body)
# print(relation_to_body)

# Length 1 Rules

In [15]:
dict_1=utils.get_r_e1e2_dict(mapped_train_data)

# Length 2 Rules

In [17]:
index_head=utils.get_head_index(mapped_train_data)
dict_2=utils.get_r1r2_e1e2_dict(mapped_train_data,index_head)

In [18]:
def get_explanation(fact,relation_to_body,dict_1,dict_2):
    pair=(fact[0],fact[2])
    rel=fact[1]
    
    if rel not in relation_to_body:
        return 0
    bodies=relation_to_body[rel]
    for body in bodies:
        if isinstance(body,int):
            if pair not in dict_1[body]:
                return 0
            return (body,-1)
        else:
            if pair not in dict_2[body]:
                return 0
            return (body,dict_2[body][pair])

In [20]:
# get_explanation((453,37,82),relation_to_body,dict_1,dict_2)

0

In [24]:
np_arr=np.loadtxt("../../prachi/kbi/kbi-pytorch/hits_1_test_fb15k")

In [39]:
arr=[]
for line in np_arr:
#     print(line)
    fact=(int(line[0]),int(line[1]),int(line[2]))
    arr.append(get_explanation(fact,relation_to_body,dict_1,dict_2))
#     print(fact)

In [40]:
# print(arr)
count=0
for x in arr:
    if x!=0:
        count+=1
print(count)

1510
